# Gridded Data Plotting

This notebook demonstrates reading and plotting gridded data including model output.

### Import Packages

In [ ]:
from datetime import datetime, timedelta

from metpy.plots import declarative
from metpy.units import units
import xarray as xr

### Get Data

In [ ]:
# Set the date/time of the model run
# Update time to within the past two weeks
date = datetime(2023, 2, 8, 0)

# Remote access to the dataset from the UCAR site
ds = xr.open_dataset('https://thredds.ucar.edu/thredds/dodsC/grib'
                     f'/NCEP/GFS/Global_onedeg/GFS_Global_onedeg_'
                     f'{date:%Y%m%d}_{date:%H%M}.grib2')

# Subset data to be just over the U.S. for plotting purposes
ds = ds.sel(lat=slice(70,10), lon=slice(360-140, 360-60))

### Plot Data

Here we demonstrate the use of `ContourPlot()` for having the computer draw contours of a variable. Only a small subset of atttributes are used in this example, but your plot can be further customized using a host of different attributes, which you can read about in the MetPy documentation.

`ContourPlot()` Documentation: https://unidata.github.io/MetPy/latest/api/generated/metpy.plots.ContourPlot.html

Additionally, the use of `BarbPlot()` is also demonstrated for plotting a vector from its components. For more details about all of the settings for plotting wind barbs the MetPy documentation has you covered.

`BarbPlot()` Documentation: https://unidata.github.io/MetPy/latest/api/generated/metpy.plots.BarbPlot.html 

In [ ]:
# Set the plot time with forecast hours
plot_time = date + timedelta(hours=0)

# Set attributes for plotting contours
cntr = declarative.ContourPlot()
cntr.data = ds
cntr.field = 'Geopotential_height_isobaric'
cntr.level = 500 * units.hPa
cntr.time = plot_time
cntr.contours = range(0, 10000, 60)
cntr.clabels = True

# Add wind barbds
barbs = declarative.BarbPlot()
barbs.data = ds
barbs.time = plot_time
barbs.field = ['u-component_of_wind_isobaric', 'v-component_of_wind_isobaric']
barbs.level = 500 * units.hPa
barbs.skip = (3, 3)
barbs.plot_units = 'knot'

# Set the attributes for the map
# and put the contours on the map
panel = declarative.MapPanel()
panel.area = [-125, -74, 22, 52]
panel.projection = 'lcc'
panel.layers = ['states', 'coastline', 'borders']
panel.title = f'500-hPa Geopotential Heights at {plot_time} by KHG'
panel.plots = [cntr, barbs]

# Set the attributes for the panel
# and put the panel in the figure
pc = declarative.PanelContainer()
pc.size = (15, 15)
pc.panels = [panel]

# Show the figure
pc.show()